In [80]:
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np


data = yf.download('GOOG', start='2020-01-01', end='2024-09-13')
#data = yf.download('AAPL', period='3mo', interval='1d')
#print(data.head())

data.isnull().sum() # if number were not 0 then
data.fillna(method='ffill', inplace=True)
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

sequence_length = 60

x = []
y = []

for i in range (sequence_length, len(data_scaled)):
    x.append(data_scaled[i - sequence_length: i, 0])
    y.append(data_scaled[i, 0])

train_size = int(len(x) * 0.8)
test_size = len(x) - train_size

x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


[*********************100%%**********************]  1 of 1 completed
/var/folders/vz/hr6jrn1n0ggf1mxbfs500y4m0000gn/T/ipykernel_1478/1876285940.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [81]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply, Flatten, BatchNormalization
import tensorflow as tf

model = Sequential()
# number of units the number of neurons
model.add(LSTM(units=50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
attention = AdditiveAttention(name='attention_weight')
model.add(Permute((2,1)))
model.add(Reshape((-1, 50)))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

#history = model.fit(x_train, y_train, epochs=100, batch_size=25, validation_split=0.2)

/Users/brianerichsenfagundes/Desktop/6017/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [82]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

tensorboard = TensorBoard(log_dir='.logs')
csv_logger = CSVLogger('training_log.csv')

callbacks_list = [early_stopping, model_checkpoint, reduce_lr, tensorboard, csv_logger]
history = model.fit(x_train, y_train, epochs=100, batch_size=25, validation_split=0.2, callbacks=callbacks_list)

Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.9373 - val_loss: 0.2032 - learning_rate: 0.0010
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.6802 - val_loss: 0.2448 - learning_rate: 0.0010
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.5783 - val_loss: 0.2599 - learning_rate: 0.0010
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3932 - val_loss: 0.2163 - learning_rate: 0.0010
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.4283 - val_loss: 0.1373 - learning_rate: 0.0010
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.5006 - val_loss: 0.1731 - learning_rate: 0.0010
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3696 - val_loss: 0.2610 - learning_rate: 0.0010
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3494 - val_loss: 0.2449 - learning_rate: 0.0010
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3370 - val_loss: 0.1479 - learning_rate: 0.0010
Epoch 10/1

In [83]:
x_test = np.array(x_test)
y_test = np.array(y_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
test_loss = model.evaluate(x_test, y_test)
print("Test loss", test_loss)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0063 
Test loss 0.009541449137032032


In [84]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = model.predict(x_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'Mean Absolute Error: {mae:.4f}')
print(f'Root Mean Squared Root: {rmse:.4f}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Mean Absolute Error: 0.0762
Root Mean Squared Root: 0.0977


/Users/brianerichsenfagundes/Desktop/6017/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [85]:
data = data[-60:]
#print(data.tail())
#print(data.head())

closing_prices = data['Close'].values

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(closing_prices.reshape(-1, 1))

x_latest = np.array([scaled_data[-60:].reshape(60)])
x_latest = np.reshape(x_latest, (x_latest.shape[0], x_latest.shape[1], 1))

predicted_stock_price = model.predict(x_latest)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)[0][0]

print(predicted_stock_price)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
161.29121


In [87]:
predicted_prices = []
current_batch = scaled_data[-60:].reshape(1, 60, 1)
for i in range(4):
    next_prediction = model.predict(current_batch)
    next_prediction_reshaped = next_prediction.reshape(1, 1, 1)
    current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)

    predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

print(predicted_prices)
print(data.tail())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[161.29121, 159.75388, 158.69054, 157.36966]
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-09-06  158.690002  159.220001  151.934998  152.130005  151.930008   
2024-09-09  153.630005  154.639999  148.199997  149.539993  149.539993   
2024-09-10  151.449997  152.300003  149.542999  150.009995  150.009995   
2024-09-11  151.089996  152.479996  148.699997  152.149994  152.149994   
2024-09-12  154.809998  155.610001  153.500000  155.539993  155.539993   

              Volume  
Date                  
2024-09-06  24999100  
2024-09-09  28057700  
2024-09-10  20401800  
2024-09-11  18991500  
2024-09-12  21024100  
